In [2]:
import os
import json
import cv2
import numpy as np

import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt

import pvlib
from datetime import datetime

# Load Data

## Load 2020 Meteorological data from Solcast

In [3]:
# solcast_data = pd.read_csv('./[NEW] Solcast_2007-2021_Hourly.csv')

In [4]:
# # filter data for year 2020
# solcast_data = solcast_data[solcast_data['Year'] == 2020]
# # filter out some columns
# solcast_data = solcast_data[['Year', 'Month', 'Day', 'Hour', 'Azimuth', 'DHI', 'DNI', 'GHI', 'Zenith']]
# # sort values by datetime
# solcast_data = solcast_data.sort_values(by=['Year', 'Month', 'Day', 'Hour'])

# solcast_data

,Year,Month,Day,Hour,Azimuth,DHI,DNI,GHI,Zenith
113928,2020,1,1,0,-143,0,0,0,168
113929,2020,1,1,1,-115,0,0,0,157
113930,2020,1,1,2,-109,0,0,0,143
113931,2020,1,1,3,-108,0,0,0,129
113932,2020,1,1,4,-109,0,0,0,116
...,...,...,...,...,...,...,...,...,...
122707,2020,12,31,19,109,0,0,0,117
122708,2020,12,31,20,108,0,0,0,130
122709,2020,12,31,21,109,0,0,0,144
122710,2020,12,31,22,116,0,0,0,158


## Load roof segment polygons

In [117]:
gdf = gpd.read_file('../Module Fitting/roof_gdf_module_fitting_flatsouth.geojson')
gdf.head()

,index,segment_id,building_id,address,image_id,category_id,num_panels,panel_area,geometry
0,0,90215,way/874972048,"C.M. Recto Street, 20 Phase 1, Bonanza, Fortun...",2986,12,1160,1898.6880,"MULTIPOLYGON (((121.12337 14.66163, 121.12339 ..."
1,1,112141,way/874972048,"C.M. Recto Street, 20 Phase 1, Bonanza, Fortun...",3707,5,888,1453.4784,"POLYGON ((121.12283 14.66167, 121.12283 14.661..."
2,2,53649,way/871081677,"128 Santan, Marikina, Metro Manila, Philippines",1775,13,740,1211.2320,"POLYGON ((121.12561 14.65969, 121.12560 14.659..."
3,3,90212,way/874972048,"C.M. Recto Street, 20 Phase 1, Bonanza, Fortun...",2986,5,734,1201.4112,"POLYGON ((121.12333 14.66191, 121.12333 14.661..."
4,4,19982,way/16827525,"Unit 2128, Riverbank Center, Riverbanks Avenue...",653,3,710,1162.1280,"POLYGON ((121.08306 14.63122, 121.08306 14.631..."


## Load orientation irradiance data

2020 hourly irradiance data (kW/m^2) per orientation (catids 2-18)

In [28]:
irradiance_orientation_df = pd.read_csv('./irradiance_orientation.csv')

In [29]:
irradiance_orientation_df

,Year,Month,Day,Hour,Azimuth,DHI,DNI,GHI,Zenith,irradiance_catid_2,...,irradiance_catid_9,irradiance_catid_10,irradiance_catid_11,irradiance_catid_12,irradiance_catid_13,irradiance_catid_14,irradiance_catid_15,irradiance_catid_16,irradiance_catid_17,irradiance_catid_18
0,2020,1,1,0,-143,0,0,0,168,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020,1,1,1,-115,0,0,0,157,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020,1,1,2,-109,0,0,0,143,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020,1,1,3,-108,0,0,0,129,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020,1,1,4,-109,0,0,0,116,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,2020,12,31,19,109,0,0,0,117,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8780,2020,12,31,20,108,0,0,0,130,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8781,2020,12,31,21,109,0,0,0,144,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8782,2020,12,31,22,116,0,0,0,158,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Generate functions for computation

In [30]:
def compute_solar_potential(panel_area, irradiance):
    """
    Parameters:
    panel_area - m^2
    irradiance - kW/m^2
    """
    module_efficiency = 0.153 # 15.3% (Google Project Sunroof)
    reduction_rate = 0.3 # 30% (Mainzer et. al., 2017)

    # compute kilowatts
    kilowatts = panel_area * irradiance

    # compute solar potential with module efficiency
    potential_with_efficiency = kilowatts * module_efficiency

    # reduce solar potential with reduction rate
    potential_reduced = potential_with_efficiency * (1.0 - reduction_rate)

    return potential_reduced

In [31]:
def add_solar_potential_to_df(roof_data, irradiance_orientation_df, output='Yearly'):
    roof_panel_area = roof_data['panel_area']
    roof_category_id = roof_data['category_id']
    roof_id = roof_data['index']

    irradiance_column = 'irradiance_catid_' + str(roof_category_id)
    single_segment_df = irradiance_orientation_df.copy()
    single_segment_df = single_segment_df[['Month','Day', 'Hour', 'Azimuth', 'DHI', 'DNI', 'GHI', 'Zenith', irradiance_column]]

    
    # generate dataframe of hourly solar potential
    if roof_panel_area > 0:
        single_segment_df['solar_potential'] = single_segment_df.apply(lambda x: compute_solar_potential(panel_area=roof_panel_area, irradiance=x[irradiance_column]), axis=1)
    else:
        single_segment_df['solar_potential'] = 0

    single_segment_df['segment_id'] = roof_id


    # aggregate data into hourly average (hour 1-23), monthly totals (1-12), and yearly
    if output == 'Hourly':
        # Hourly Average
        solar_potential_hourly_avg = single_segment_df.groupby(by=['Hour'])['solar_potential'].mean().to_frame()
        solar_potential_hourly_avg['segment_id'] = roof_id
        solar_potential_hourly_avg['building_id'] = roof_data['building_id']
        solar_potential_hourly_avg['Hour'] = solar_potential_hourly_avg.index

        return solar_potential_hourly_avg
  
    elif output == 'Monthly':
        # Monthly Totals
        solar_potential_monthly = single_segment_df.groupby(by=['Month'])['solar_potential'].sum().to_frame()
        solar_potential_monthly['segment_id'] = roof_id
        solar_potential_monthly['building_id'] = roof_data['building_id']
        solar_potential_monthly['Month'] = solar_potential_monthly.index

        return solar_potential_monthly

    elif output == 'Yearly':
        # Yearly Total
        yearly_value = single_segment_df['solar_potential'].sum()
        solar_potential_yearly = pd.DataFrame([[roof_id, yearly_value]], columns = ['segment_id', 'solar_potential'])
        solar_potential_yearly['building_id'] = roof_data['building_id']

        return solar_potential_yearly

    else:
        return None

# Run functions on data and combine into a single dataframe

## Hourly Data (24 values each)

In [32]:
# run lambda function to generate individual dataframes
hourly_results = gdf.apply(lambda x: add_solar_potential_to_df(x, irradiance_orientation_df, output='Hourly'), axis=1)

# concatenate individual dataframes into one single dataframe
hourly_solar_potential_df = pd.concat([r for r in hourly_results], ignore_index=True)
hourly_solar_potential_df.reset_index(drop=True)

# export file
hourly_solar_potential_df.to_csv('hourly_solar_potential_df.csv')

## Monthly Data (12 values each)

In [34]:
# run lambda function to generate individual dataframes
monthly_results = gdf.apply(lambda x: add_solar_potential_to_df(x, irradiance_orientation_df, output='Monthly'), axis=1)

# concatenate individual dataframes into one single dataframe
monthly_solar_potential_df = pd.concat([r for r in monthly_results], ignore_index=True)
monthly_solar_potential_df.reset_index(drop=True)

# download file
monthly_solar_potential_df.to_csv('monthly_solar_potential_df.csv')

## Annual Data (1 value each)

In [36]:
# run lambda function to generate individual dataframes
annual_results = gdf.apply(lambda x: add_solar_potential_to_df(x, irradiance_orientation_df, output='Yearly'), axis=1)

# concatenate individual dataframes into one single dataframe
annual_solar_potential_df = pd.concat([r for r in annual_results], ignore_index=True)
annual_solar_potential_df.reset_index(drop=True)

# download file
annual_solar_potential_df.to_csv('annual_solar_potential_df.csv')